# Practical 2: Mobile Robots

We will import the libraries in the following blocks.

In [ ]:
from Practical02_Support.Renderer import *
import ipywidgets as widgets
import numpy as np

from ece4078.Utility import StartMeshcat

In [ ]:
vis = StartMeshcat()

# 1. Bicycle Kinematic Model

The configuration (also known as state) of the bycycle is descibed by its 2D position and orientation, i.e., $(x, y, \theta)$. The control inputs are the speed at the rear wheel and the steering angle at the front wheel, i.e., $(v, \delta)$

In the following we implement the bicycle kinematic model and show how its motion is described in the world frame.

In [ ]:
class Bicycle(bot2D):
    """Implementation of the kinematic bicycle model (rear wheel model)
    length = distance (in meters) between the wheels
    gamma_max = maximum steering angle
    speed_max = maximum speed
    """

    def __init__(self, length=1, delta_max=np.pi/3, vel_max=5):
        super().__init__()
        
        # Length between the wheels
        self.length = length
        
        # Maximum velocity and steering angle
        self.delta_max = delta_max
        self.vel_max = vel_max
        
        # Initial control inputs
        self.velocity = 0
        self.delta = 0
        
    def update_control(self, v, delta):
        self.velocity = np.clip(v, -self.vel_max, self.vel_max)
        self.delta = np.clip(delta, -self.delta_max, self.delta_max)
        
    def drive(self, dt=0.01):
        """
        Update the bicycle's state.
        """        
        # Compute velocity using control input and current orientation
        v_x = self.velocity * np.cos(self.theta)
        v_y = self.velocity * np.sin(self.theta)
        
        # Compute next position
        next_x = self.x + v_x * dt
        next_y = self.y + v_y * dt
                
        # Compute angular velocity given control input (velocity and
        # steering angle)
        omega = self.velocity * np.tan(self.delta) / self.length
        
        # Compute next orientation
        next_theta = self.theta + omega * dt
        self.set_state(next_x, next_y, next_theta)
        
    def reset(self):
        super().reset()
        self.velocity = 0
        self.delta = 0

## Lets now test driving the bicycle
- Use the slider to adjust the velocity and steering angle. 
- Currently the animation will play for 180 frames. You can adjust this number by providing the `max_iterations` argument to the function call `rend.initialize(bot, max_iterations=[integer])`

In [ ]:
max_iterations = 180
max_vel = 5         # robot's maximum velocity
max_delta = np.pi/3 # robot's maximum steering angle
bike_length = 0.25  # create a bicycle with .25 m between the rear and front wheel
robot_start_x = -2.8 # robot's starting x position
robot_start_y = -2.8 # robot's starting y position
robot_start_theta = np.pi/6 # robot's starting orientation
time_step = 0.05

vis.delete()
vis.Set2DView(scale = 5)
vis.mask_origin()
bot = Bicycle(bike_length, max_delta, max_vel)
bot.set_state(robot_start_x, robot_start_y, robot_start_theta)

rend = Renderer.Instance()
rend.initialize(vis, bot, dt=time_step, max_iterations=max_iterations, realtime = True)
rend.spawn_robot()

In [ ]:
v_slider = widgets.FloatSlider(value=0, min=-max_vel, max=max_vel, step=0.1, 
                               description=r'Velocity (v)', readout_format='.1f')
delta_slider = widgets.FloatSlider(value=0, min=-max_delta, max=max_delta, step=0.05,
                                   description=r'Angle (delta)', readout_format='.1f')
ui = widgets.HBox([v_slider, delta_slider])
control_widgets = widgets.interactive_output(bot.update_control, {'v': v_slider, 'delta': delta_slider})
display(ui, control_widgets)

rend.show_control_panel()
def reset_bicycle(b = None):
    rend.reset()
    v_slider.value = 0
    delta_slider.value = 0
rend.btn_reset.on_click(reset_bicycle)
rend.start_render_loop()
display(vis.show_inline(height = 500))

# 2. PenguinPi Kinematic Model

<img src=https://i.postimg.cc/MGKJr92G/TvZ7gVN.jpg width="340" height="300">


The state of the robot is described by its 2D position $(x,y)$  and orientation $\theta$. Our control inputs are the linear and angular velocity of the robot, i.e., $v$ and $\omega$. 

We will define the <code>drive(self, dt=0.02)</code> function that will update the state of the robot given a sequence of control inputs.

**Interaction**
- Move the sliders to change the control inputs

**TODO**
- Verify code correctness 
- Complete the code that makes the robot turn

In [ ]:
class PenguinPi(bot2D):

    def __init__(self, max_v=5, max_omega=np.pi):
        super().__init__()
        
        # Control inputs
        self.linear_velocity = 0
        self.angular_velocity = 0
        
        # Control input bounds
        self.max_linear_velocity = max_v
        self.max_angular_velocity = max_omega
        
    def update_control(self, linear_v, angular_v):
        # We read the values from the sliders and update the robot's linear and angular velocity
        self.linear_velocity = np.clip(linear_v, -self.max_linear_velocity, self.max_linear_velocity)
        self.angular_velocity = np.clip(angular_v, -self.max_angular_velocity , self.max_angular_velocity)
        
    def drive(self, dt=0.02):
        """
         Update the PenguiPi state
        """        

        # Remember that the PenguiPi current state is given by self.x, self.y, self.theta        
        # Apply the velocities
        if self.angular_velocity == 0:

            next_x = self.x - np.cos(self.theta)*self.linear_velocity*dt
            next_y = self.y + np.sin(self.theta)*self.linear_velocity*dt
            next_theta = self.theta
        else:
            #TODO: Complete code that make the robot turn-----------------------------
            R = 0
            next_theta = self.theta
            next_x = self.x
            next_y = self.y
            #ENDTODO ------------------------------------------------------------------

        # Make next state our current state
        self.set_state(next_x, next_y, next_theta) 
        
    def reset(self):
        super().reset()
        self.linear_velocity = 0
        self.angular_velocity = 0

## Lets now test driving the PenguiPi
- Use the slider to adjust the velocity and steering angle
- Currently the animation will play for 180 frames. You can adjust this number by providing the ```max_iterations``` argument to the function call ```rend.initialize(bot, max_iterations=[integer])```

In [ ]:
max_iterations = 180 # maximum number of animation frames
max_linear_vel = 5
max_angular_vel = np.pi
pibot_start_x = -3.0 # pibot's starting x position
pibot_start_y = -3.0 # pibot's starting y position
pibot_start_theta = np.pi/6 # pibot's starting orientation
time_step = 0.05

vis.delete()
vis.Set2DView(scale = 5)
vis.mask_origin()
pi_bot = PenguinPi(max_linear_vel, max_angular_vel)
pi_bot.set_state(pibot_start_x, pibot_start_y, pibot_start_theta)

rend = Renderer.Instance()
rend.initialize(vis, pi_bot, dt=time_step, max_iterations=max_iterations, realtime = True)
rend.spawn_robot()

In [ ]:
linear_slider = widgets.FloatSlider(value=0, min=-max_linear_vel, max=max_linear_vel, step=0.1, 
                                    description=r'v', readout_format='.1f')
angular_slider = widgets.FloatSlider(value=0, min=-max_angular_vel, max=max_angular_vel, step=0.1,
                                     description=r'omega', readout_format='.1f')
ui = widgets.HBox([linear_slider, angular_slider])
control_widgets = widgets.interactive_output(
    pi_bot.update_control, {'linear_v': linear_slider, 'angular_v': angular_slider})
display(ui, control_widgets)

rend.show_control_panel()
def reset_penguinpi(b = None):
    rend.reset()
    linear_slider.value = 0
    angular_slider.value = 0
rend.btn_reset.on_click(reset_penguinpi)
rend.start_render_loop()
display(vis.show_inline())